In [92]:
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error


import numpy as np

In [4]:
movies_df = pd.read_csv('ml-1m/movies.dat',
                        delimiter='::', engine= 'python', header=None,
                        names=['movie_name', 'genre'], encoding = "ISO-8859-1")

In [5]:
rating_df = pd.read_csv('ml-1m/ratings.dat',
                        delimiter='::', engine= 'python', header=None,
                        names=['user_id', 'Movie_Id','Ratings','Time_stamp'], encoding = "ISO-8859-1")

In [6]:
users_df = pd.read_csv('ml-1m/users.dat',
                        delimiter='::', header=None,
                        names=['user_id', 'Gender','Age','Occupation','Zip-Code'], encoding = "ISO-8859-1")

C:\Users\aczer\AppData\Local\Temp\ipykernel_14208\3957394905.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  users_df = pd.read_csv('ml-1m/users.dat',


In [70]:
# rating_df.loc[2]['movie_id']

In [34]:
j = 0
for i in rating_df[:]:
    print(i)
    j+= 1 
    if j > 30:
        break

user_id
Movie_Id
Ratings
Time_stamp


In [42]:
len(rating_df)

1000209

In [80]:
def cross_validation(df,n_folds):
    shuffled_df = df.sample(random_state = 42, frac =1)
    shuffled_df['Fold']= None
    shuffled_df.reset_index(inplace = True)
    shuffled_df.drop(columns = 'index', inplace = True)
    data_size = len(shuffled_df)
    for i in range(1,n_folds):
        shuffled_df.loc[int((i-1)/n_folds*data_size):int(i/n_folds * data_size),'Fold'] = i
    shuffled_df.loc[int((n_folds-1)/n_folds*data_size):,'Fold']= n_folds
    return shuffled_df

In [81]:
data = cross_validation(rating_df, 5)

In [104]:
# data.loc[data['Fold']== 5]
data

,user_id,Movie_Id,Ratings,Time_stamp,Fold
0,5412,2683,2,960243649,1
1,5440,904,5,959995181,1
2,368,3717,4,976311423,1
3,425,1721,4,976283587,1
4,4942,3697,1,962642480,1
...,...,...,...,...,...
1000204,1586,1077,5,974735719,5
1000205,2129,2700,5,974643199,5
1000206,854,3102,3,975355597,5
1000207,4033,3479,5,965525805,5


In [84]:
data.reset_index(inplace = True)

In [110]:
class MatrixFactorization:
    def __init__(self,user_size,item_size):
        #initilaze two matrixes that then multiply by each other to give a matrix of ratings
        self.user_matrix = np.random.rand(user_size,10)
        self.item_matrix = np.random.rand(item_size,10)
    def fit(self,x, learning_rate = 0.005, lamb = 0.05, n_iter = 2):
        for it in range(n_iter):
            for i in range(len(x)):
                user = x.loc[i]['user_id']
                item = x.loc[i]['Movie_Id']

                #calculate the error
                error = x.loc[i]['Ratings'] - np.matmul(self.user_matrix[user],self.item_matrix[item])
                # update values
                self.user_matrix[user] = self.user_matrix[user] + learning_rate*(error*self.item_matrix[item] - lamb*self.user_matrix[user])
                self.item_matrix[item] = self.item_matrix[item] + learning_rate*(error*self.user_matrix[user] - lamb*self.item_matrix[item])
        print('current iteration ended: '+str(it))
    def test(self,x):
        predictions = []
        for i in range(len(x)):
            user = x.loc[i]['user_id']
            item = x.loc[i]['Movie_Id']

            #calculate the error
            pred = np.matmul(self.user_matrix[user],self.item_matrix[item])
            predictions.append(pred)
        labels = np.array(x['Ratings'])
        predictions = np.array(predictions)
        rmse =  mean_squared_error(labels,predictions, squared = False)
        mse = mean_absolute_error(labels,predictions)

        return rmse, mse

In [96]:
mt = MatrixFactorization(6041,3953)

In [98]:
%%time
mt.fit(rating_df,n_iter = 1 )

KeyboardInterrupt: 

In [91]:
np.array(data['Ratings'])

array([2, 5, 4, ..., 3, 5, 4], dtype=int64)

In [69]:
mt.user_matrix root mean square error and MAE

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.20888081, 0.20888081, 0.20888081, ..., 0.20888081, 0.20888081,
        0.20888081],
       [0.09690053, 0.09690053, 0.09690053, ..., 0.09690053, 0.09690053,
        0.09690053],
       ...,
       [0.99865908, 0.99865908, 0.99865908, ..., 0.99865908, 0.99865908,
        0.99865908],
       [0.96198413, 0.96198413, 0.96198413, ..., 0.96198413, 0.96198413,
        0.96198413],
       [0.79187219, 0.79187219, 0.79187219, ..., 0.79187219, 0.79187219,
        0.79187219]])

In [112]:
c_rmse = 10000
user_params = None
item_params = None
rmse_list = []
mae_list = []
for i in range(1,6):
    train = data.loc[data['Fold'] != i ]
    # print(train)
    train = train.reset_index()
    valid = data.loc[data['Fold'] == i ] 
    valid = valid.reset_index()
    mt = MatrixFactorization(6041,3953)
    # print(train)
    print('Fitting the fold: ' + str(i))
    mt.fit(train)
    rmse, mae = mt.test(valid)
    print(f"Fold {i} RMSE: {rmse} \n MSE: {mae}")
    rmse_list.append(rmse)
    mae_list.append(mae)
print(f'Mean results:\nRMSE: {np.mean(np.array(rmse_list))} \nMAE: {np.mean(np.array(mae_list))}')
    # if rmse < c_rmse:
    #     user_params = mt.user_matrix
    #     item_params = mt.item_matrix

Fitting the fold: 1
current iteration ended: 1
Fold 1 RMSE: 0.9348465562690712 
 MSE: 0.7452372221719877
Fitting the fold: 2
current iteration ended: 1
Fold 2 RMSE: 0.9299440091706563 
 MSE: 0.7406167068105701
Fitting the fold: 3
current iteration ended: 1
Fold 3 RMSE: 0.9292214862910607 
 MSE: 0.7395162109234338
Fitting the fold: 4
current iteration ended: 1
Fold 4 RMSE: 0.9313222016108349 
 MSE: 0.7411128144181096
Fitting the fold: 5
current iteration ended: 1
Fold 5 RMSE: 0.9320634503794772 
 MSE: 0.7406185208757
Mean results:
RMSE: 0.9314795407442201 
MAE: 0.7414202950399602


In [84]:
data

,user_id,Movie_Id,Ratings,Time_stamp,Fold
0,5412,2683,2,960243649,1
1,5440,904,5,959995181,1
2,368,3717,4,976311423,1
3,425,1721,4,976283587,1
4,4942,3697,1,962642480,1
...,...,...,...,...,...
1000204,1586,1077,5,974735719,5
1000205,2129,2700,5,974643199,5
1000206,854,3102,3,975355597,5
1000207,4033,3479,5,965525805,5
